In [5]:
%reset -f

from functools import reduce
import random
import math
import numpy as np
from itertools import combinations
from itertools import product
from itertools import combinations_with_replacement
from datetime import datetime

from stats import Stats
from character import Character

# File path and name
file_path = "res.txt"
SLOT_NUMER = 37

#============getting all the items from the db.csv file=========================#
items_in_slot = {k:[] for k in range(SLOT_NUMER)}
import pandas as pd
df=pd.read_csv('db.csv')
for index, row in df.iterrows():
    d = row.dropna().to_dict()
    
    items_in_slot[d['slot']].append(Stats(d['slot'], d['name'], d['set'],attributes=d))

#=======filter items_in_slot and prepare for the combination function===========#
items_in_slot_filtered = list(filter(None, items_in_slot.values()))

#================print some info about the objects loaded=======================#
for slot, item in items_in_slot.items():
    if len(item) > 0:
        print(f'slot: {slot:3}, choices:{len(item)}')
items_in_slot_filtered_len = reduce(lambda a, b: a*len(b), [1,*items_in_slot_filtered])
print(f'Items combinations: {items_in_slot_filtered_len}')


#==============base stats for a specific class (not present in db)==============#
base_class_stats    = { 'phy_atk':1202,	'mag_atk':1202,	'phy_def':301,	'mag_def':301,	'hp':157000,
                        'dmg_per':      18,  
                        'dmg_per_m':    0,
                        'crit':         25,        
                        'maxi':         0,        
                        'crit_dmg':     25,

                        'pola':         0,
                        'specific':     0,
                        'all_s_dmg':    3,
                        'boss_dmg':     0,
                        'adapt':        0,
                        'cdr':          0
                       }

#===============================force passives==================================#
head_hunter = {'dmg_per':-24, 'boss_dmg':80}


#============================sockets combinations===============================#
available_sockets = [  
                        Stats('0', 'socket', attributes={'maxi':12}),
                        Stats('0', 'socket', attributes={'crit':12}),
                        Stats('0', 'socket', attributes={'boss_dmg':5})
                    ]

sockets_combinations = list(combinations_with_replacement(available_sockets, 19))
sockets_combinations_len = len(sockets_combinations)
print(f'Socket combinations: {sockets_combinations_len}')

# generate item for each socket combination, grouping sockets into a single item
itemized_sockets = []
for i, sockets in enumerate(sockets_combinations):
    tot_socket = Stats('0', 'SOCKETS')
    for socket in sockets:
        tot_socket += socket
    itemized_sockets.append(tot_socket)


#=========================combinatory for all items=============================#
print('Combinations, Starting')
combs = np.array(np.meshgrid(*items_in_slot_filtered, itemized_sockets)).T.reshape(-1,len([*items_in_slot_filtered, itemized_sockets]))
#combs = list(product(*list(items_in_slot_filtered), itemized_sockets))

#combs = random.sample(combs, 100000)
#combs = combs[::500]
combs_len = len(combs)
print(f'Total combinations: {combs_len}')

print(f'Total combinations as expected: {combs_len==(items_in_slot_filtered_len*sockets_combinations_len)}')
print('Combinations, Complete')

#=============================create specific items ============================#
base_class_stats_item = Stats('base','class base stats','',attributes=base_class_stats)
head_hunter_force_passive = Stats('HH','unique HH','',attributes=head_hunter)



#===========equip items combinations and specific items if needed===============#
print('Item placement and damage calculation, Starting')

new_character = Character('Char', '')
new_character = new_character.equip(base_class_stats_item, base_class_stats_item.id)
new_character = new_character.equip(head_hunter_force_passive, head_hunter_force_passive.id)

mul_list = []
for i, items in enumerate(combs):
    # create a new "character" for each item combination
    new_character.name = 'jeff_%s'%i
    new_character.items = {}
    new_character.attributes = {}
    
     # equip all items
    for item in items:
        new_character = new_character.equip(item, item.id)
    
    # save to list
    mul_list.append(new_character.get_damage()[1])
    
    

print('Item placement and damage calculation, Complete')

#==============apply the damage calculation to each character===================#
#dataset = []
#for x_jeff in chars:
#	mul, dmg, x_jeff = x_jeff.get_damage()
#	dataset.append({'factor':math.trunc(mul,4), 'damage':math.trunc(dmg,2), 'obj':x_jeff})

#mul_list = [  for x_jeff in chars]

# and sort ascending

best_comb_index = mul_list.index(max(mul_list))

best_character = Character('Char', '')
best_character = new_character.equip(base_class_stats_item, base_class_stats_item.id)
best_character = new_character.equip(head_hunter_force_passive, head_hunter_force_passive.id)

for item in combs[best_comb_index]:
    best_character = best_character.equip(item, item.id)

best = new_character.get_damage()




#=========================save to file best N entries===========================#
with open(file_path, "w") as file:
    file.write(f'Latest update: {datetime.now()}\n')
    
    output_string = f"{math.trunc(best[0]):<10}{math.trunc(best[1]):<10}\n{best[2]}\n"
    
    file.write(output_string.replace('|0.0','|   '))
    
print('DONE')

slot:   2, choices:2
slot:   3, choices:4
slot:   4, choices:4
slot:   5, choices:4
slot:   6, choices:4
slot:   7, choices:2
slot:   8, choices:2
slot:   9, choices:2
slot:  10, choices:2
slot:  11, choices:2
slot:  14, choices:2
slot:  15, choices:1
slot:  19, choices:1
slot:  20, choices:3
slot:  22, choices:1
Items combinations: 98304
Socket combinations: 210
Combinations, Starting
Total combinations: 20643840
Total combinations as expected: True
Combinations, Complete
Item placement and damage calculation, Starting


KeyboardInterrupt: 

In [ ]:

mul_list

In [69]:
a = [[1,5,4,8],[2,3,4,8]]
b = [78,54,21,32,65]
c = *a,b
print(c)
np.array(np.meshgrid(*a,b))


max([1,2,5])

([1, 5, 4, 8], [2, 3, 4, 8], [78, 54, 21, 32, 65])


5